# Iterative Prompt Development
In this lesson, you'll iteratively analyze and refine your prompts to generate marketing copy from a product fact sheet.

## Setup

In [1]:
import json
import boto3
import panel as pn  # GUI

bedrock_runtime = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1",
)

In [2]:
def get_completion(prompt, modelId="ai21.jamba-1-5-mini-v1:0"):
    messages = [{"role": "user", "content": prompt}]
    body = {
        "messages": messages,
        "temperature": 0
    }
    response = bedrock_runtime.invoke_model(
        body=json.dumps(body),
        modelId=modelId
    )
    response_body = json.loads(response.get("body").read())
    return response_body.get("choices")[0]["message"]["content"]


## Generate a marketing product description from a product fact sheet¶

In [3]:
fact_sheet_chair = """
OVERVIEW
- Part of a beautiful family of mid-century inspired office furniture, 
including filing cabinets, desks, bookcases, meeting tables, and more.
- Several options of shell color and base finishes.
- Available with plastic back and front upholstery (SWC-100) 
or full upholstery (SWC-110) in 10 fabric and 6 leather options.
- Base finish options are: stainless steel, matte black, 
gloss white, or chrome.
- Chair is available with or without armrests.
- Suitable for home or business settings.
- Qualified for contract use.

CONSTRUCTION
- 5-wheel plastic coated aluminum base.
- Pneumatic chair adjust for easy raise/lower action.

DIMENSIONS
- WIDTH 53 CM | 20.87”
- DEPTH 51 CM | 20.08”
- HEIGHT 80 CM | 31.50”
- SEAT HEIGHT 44 CM | 17.32”
- SEAT DEPTH 41 CM | 16.14”

OPTIONS
- Soft or hard-floor caster options.
- Two choices of seat foam densities: 
 medium (1.8 lb/ft3) or high (2.8 lb/ft3)
- Armless or 8 position PU armrests 

MATERIALS
SHELL BASE GLIDER
- Cast Aluminum with modified nylon PA6/PA66 coating.
- Shell thickness: 10 mm.
SEAT
- HD36 foam

COUNTRY OF ORIGIN
- Italy
"""

In [4]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)


 **Product Description:**

Introducing the SWC-100/110 Office Chair, a stunning addition to any workspace, inspired by the timeless elegance of mid-century design. This versatile chair is perfect for both home and business settings, offering a range of customization options to suit your personal style and functional needs.

Choose between two upholstery options: plastic back and front (SWC-100) or full upholstery (SWC-110), available in ten fabric and six leather choices. The chair's base is available in stainless steel, matte black, gloss white, or chrome finishes, allowing you to create a look that complements your decor. Additionally, you can opt for armrests or a sleek armless design, and select from two seat foam densities—medium (1.8 lb/ft3) or high (2.8 lb/ft3)—for the ultimate in comfort.

Constructed with a durable 5-wheel plastic-coated aluminum base and featuring a pneumatic chair adjust for easy height control, this chair ensures both stability and ease of use. Its dimensio

## Issue 1: The text is too long 
- Limit the number of words/sentences/characters.

In [5]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)


 Experience the elegance of mid-century design with our versatile office chair. Choose from various shell colors, base finishes, and upholstery options to match your style. Perfect for any setting, this chair features a 5-wheel base, pneumatic height adjustment, and high-density foam for ultimate comfort. Made in Italy.


In [6]:
len(response.split())

47

## Issue 2. Text focuses on the wrong details
- Ask it to focus on the aspects that are relevant to the intended audience.

In [7]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

 Crafted in Italy, this office chair features a cast aluminum shell with a modified nylon PA6/PA66 coating, ensuring durability and a sleek finish. The seat is constructed from high-density HD36 foam, providing exceptional comfort. The 5-wheel base is made of plastic-coated aluminum, complemented by a pneumatic chair adjust for smooth height adjustment.


In [8]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

At the end of the description, include every 7-character 
Product ID in the technical specification.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

 The SWC-100 office chair features a cast aluminum shell with a modified nylon PA6/PA66 coating, ensuring durability and a sleek finish. Its seat is constructed from high-density HD36 foam, providing comfort and support. The chair is mounted on a 5-wheel base made of plastic-coated aluminum, with options for soft or hard-floor casters. Product ID: 20871.


## Issue 3. Description needs a table of dimensions
- Ask it to extract information and organize it in a table.

In [9]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

At the end of the description, include every 7-character 
Product ID in the technical specification.

After the description, include a table that gives the 
product's dimensions. The table should have two columns.
In the first column include the name of the dimension. 
In the second column include the measurements in inches only.

Give the table the title 'Product Dimensions'.

Format everything as HTML that can be used in a website. 
Place the description in a <div> element.

Technical specifications: ```{fact_sheet_chair}```
"""

response = get_completion(prompt)
print(response)

 <div>
  <p>
    This mid-century inspired office chair is constructed with a cast aluminum shell featuring a modified nylon PA6/PA66 coating, ensuring durability and a sleek finish. The shell boasts a thickness of 10 mm, providing robust support. The seat is crafted from high-density HD36 foam, offering exceptional comfort and longevity. The chair is mounted on a 5-wheel base made of plastic-coated aluminum, ensuring smooth mobility. A pneumatic chair adjust allows for effortless height adjustment.
  </p>
  <p>
    Product IDs: SWC-100, SWC-110.
  </p>
  <h2>Product Dimensions</h2>
  <table>
    <thead>
      <tr>
        <th>Dimension</th>
        <th>Measurements</th>
      </tr>
    </thead>
    <tbody>
      <tr>
        <td>Width</td>
        <td>53 CM | 20.87”</td>
      </tr>
      <tr>
        <td>Depth</td>
        <td>51 CM | 20.08”</td>
      </tr>
      <tr>
        <td>Height</td>
        <td>80 CM | 31.50”</td>
      </tr>
      <tr>
        <td>Seat Height</td>
        

## Load Python libraries to view HTML

In [10]:
from IPython.display import display, HTML

In [11]:
display(HTML(response))

Dimension,Measurements
Width,53 CM | 20.87”
Depth,51 CM | 20.08”
Height,80 CM | 31.50”
Seat Height,44 CM | 17.32”
Seat Depth,41 CM | 16.14”


## Try experimenting on your own!